In [ ]:
from datasets import load_dataset
import pandas as pd
import re

from zhipuai import ZhipuAI

client = ZhipuAI(api_key="") # APIKey
def call_llm_api(prompt, model="glm-4-air", temperature=0.5):
    response = client.chat.completions.create(
    model = model,
    temperature = temperature,
    messages=[
        {
                    "role": "system",
                    "content": (
                        "You are a helpful assistant. Translate the following question into predicate logic, but reserve the meaningful word here, and use the format like ∃x Invented(x, Bulb). Respond as concisely as possible."
                    ),
                },
        {"role": "user", "content": prompt},
    ],
)
    return (response.choices[0].message.content)

# Load the dataset
ds = load_dataset("lucadiliello/triviaqa")

df = pd.DataFrame(ds['train'][:5500])

# Filter out rows with more than one question mark
df = df[~df['question'].str.contains(r'\?.*\?')]

# Ensure that each remaining row has only one question mark
df['question'] = df['question'].str.replace(r'\?+', '?', regex=True)

def process_questions(df, output_path):
    for index, row in df.iterrows():
        logical_form = call_llm_api(row['question'])
        df.at[index, 'logical_form'] = logical_form
        df.to_csv(output_path,  encoding='utf-8-sig', index=False)

output_csv_path = 'train.csv'

process_questions(df, output_csv_path)